In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio

import dataloader
import bandpower

# Extract stress from excel

In [2]:
df = pd.read_csv('./data/SFC_DASS21.csv')
df.head()

,session,semester,subject,depression,anxiety,stress
0,'20140922_s1',1,1,18,20,38
1,'20141103_s1+s2',1,1,28,28,30
2,'20141201_s1+s2',1,1,34,34,32
3,'20141215_s1+s2',1,1,26,24,22
4,'20141229_s1+s2',1,1,36,26,30


In [3]:
df['stress'].describe()

count    171.000000
mean       7.450292
std        8.510315
min        0.000000
25%        0.000000
50%        4.000000
75%       12.000000
max       38.000000
Name: stress, dtype: float64

In [4]:
df[df['subject']<19]

,session,semester,subject,depression,anxiety,stress
0,'20140922_s1',1,1,18,20,38
1,'20141103_s1+s2',1,1,28,28,30
2,'20141201_s1+s2',1,1,34,34,32
3,'20141215_s1+s2',1,1,26,24,22
4,'20141229_s1+s2',1,1,36,26,30
...,...,...,...,...,...,...
89,'20141002_s17+s18',1,18,2,2,0
90,'20141023_s17+s18',1,18,0,2,0
91,'20141127_s17+s18',1,18,2,4,4
92,'20141211_s17+s18',1,18,2,2,0


In [8]:
# Get increased stress or normal label for each subject
num_sub = 26
group_mean = df.groupby(by='subject').mean()
labels = []

for i in range(len(df)):
    stress_sub = df.iloc[i]['stress']
    sub = df.iloc[i]['subject']
    threshold = group_mean.loc[sub]['stress']
    
    labels.append(stress_sub>threshold)
    
labels = np.asarray(labels, 'int')
df['label'] = labels
print('Label 1: %d, Label 0: %d'%(np.sum(labels), len(labels)-np.sum(labels)))

Label 1: 58, Label 0: 113


In [9]:
df.head()

,session,semester,subject,depression,anxiety,stress,label
0,'20140922_s1',1,1,18,20,38,1
1,'20141103_s1+s2',1,1,28,28,30,0
2,'20141201_s1+s2',1,1,34,34,32,1
3,'20141215_s1+s2',1,1,26,24,22,0
4,'20141229_s1+s2',1,1,36,26,30,0


## Extract EEG data from mat files

In [14]:
A, B, C = dataloader.read_data()

Load data from .mat files...


In [25]:
channels = sio.loadmat('./data/ch_lib.mat')
df = pd.DataFrame(columns = ['ID'] + list(range(30)))

for i in range(len(channels['ch_lib'])):
    channels_i = channels['ch_lib'][i][0][0]
    channels_select_i = [channels_i[i][0] for i in range(len(channels_i))]
    num_channel_i = len(channels_select_i)
    df.loc[i] = [i+1] + list(channels_select_i) + ['None']*(30-num_channel_i)
    

In [26]:
df

,ID,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,1,C3,C4,CP3,CP4,CPZ,CZ,F3,F4,F7,...,OZ,P3,P4,PZ,T3,T4,T5,T6,TP7,TP8
1,2,C3,C4,CP3,CP4,CPZ,CZ,F3,F4,F7,...,OZ,P3,P4,PZ,T3,T4,T5,T6,TP7,TP8
2,3,C3,C4,CP3,CP4,CPZ,CZ,F3,F4,F7,...,OZ,P3,P4,PZ,T3,T4,T5,T6,TP7,TP8
3,4,C3,C4,CP3,CP4,CPZ,CZ,F3,F4,F7,...,P3,P4,PZ,T3,T4,T5,T6,TP7,TP8,None
4,5,C3,C4,CP3,CP4,CPZ,CZ,F3,F4,F7,...,OZ,P3,P4,PZ,T3,T4,T5,T6,TP7,TP8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,167,C3,C4,CP3,CP4,CPZ,CZ,F3,F4,F7,...,OZ,P3,P4,P7,P8,PZ,T7,T8,TP7,TP8
167,168,C3,C4,CP3,CP4,CPZ,CZ,F3,F4,F7,...,OZ,P3,P4,P7,P8,PZ,T7,T8,TP7,TP8
168,169,C3,C4,CP3,CP4,CPZ,CZ,F3,F4,F7,...,OZ,P3,P4,P7,P8,PZ,T7,T8,TP7,TP8
169,170,C3,C4,CP3,CP4,CPZ,CZ,F3,F4,F7,...,OZ,P3,P4,P7,P8,PZ,T7,T8,TP7,TP8


In [6]:
channels_i = channels['ch_lib'][0][0][0]
channels_select_i = [channels_i[i][0] for i in range(len(channels_i))]
channels_select_i

['C3',
 'C4',
 'CP3',
 'CP4',
 'CPZ',
 'CZ',
 'F3',
 'F4',
 'F7',
 'F8',
 'FC3',
 'FC4',
 'FCZ',
 'FP1',
 'FP2',
 'FT7',
 'FT8',
 'FZ',
 'O1',
 'O2',
 'OZ',
 'P3',
 'P4',
 'PZ',
 'T3',
 'T4',
 'T5',
 'T6',
 'TP7',
 'TP8']

In [4]:
dataloader.find_common_channels()

['C4', 'CP3', 'CP4', 'F7', 'F8', 'FC4', 'FZ', 'P3']

# Extract DSS Stress level

In [26]:
EEG_list, labels, df_all = dataloader.read_data()

Load data from .mat files...


In [29]:
df_all.columns

Index(['session', 'semester', 'subject', 'depression', 'anxiety', 'stress',
       'number', 'taskLabel', 'purpose', 'session', 'subject',
       'inSessionNumber', 'gender', 'YOB', 'age', 'hand', 'vision', 'hearing',
       'height', 'weight', 'medication', 'Unnamed: 15', 'channelLocations',
       'channelLocationType', 'dataRecording', 'Unnamed: 19', 'Unnamed: 20',
       'modality', 'NA', 'NA.1', 'NA.2', 'NA.3', 'NA.4', 'NA.5', 'NA.6',
       'channels'],
      dtype='object')

In [33]:
df_all['channelLocations']

0        30ch_loc_NuAmps.xls
1        30ch_loc_NuAmps.xls
2        30ch_loc_NuAmps.xls
3        30ch_loc_NuAmps.xls
4        30ch_loc_NuAmps.xls
               ...          
165      30ch_loc_NuAmps.xls
166    30ch_loc_SynAmps2.xls
167    30ch_loc_SynAmps2.xls
168      30ch_loc_NuAmps.xls
169      30ch_loc_NuAmps.xls
Name: channelLocations, Length: 170, dtype: object

In [57]:
subjects = list(range(1,27))
df_subs = pd.DataFrame()

for subject in subjects:
    if subject <= 18:
        file_path = './data/DSS/first semester/%s/first_datebystress%d.txt'%(str(subject).zfill(2), subject)
    else:
        file_path = './data/DSS/second semester/%s/second_datebystress%d.txt'%(str(subject-18).zfill(2), subject-18)

    df = pd.read_csv(file_path, sep = '\s+', header=0, names=['date','stress'], index_col=False)
    df['subject'] = [subject]*len(df)
    df_subs = df_subs.append(df)
    
df_subs = df_subs.reset_index(drop=True)

In [64]:
df_subs.groupby(by = 'subject').count()

,date,stress
subject,,
1,219,219
2,79,79
3,90,90
4,86,86
5,107,107
6,112,112
7,108,108
8,118,118
9,86,86


In [80]:
sub_df = pd.read_csv('./data/SFC_DASS21.csv')

In [83]:
session_list = sub_df['session'].values

for i in range(len(sub_df)):
    date = session_list[i]
    date = date.split('_')[0][1:]
    date = date[0:4] + '-' + date[4:]
    session_list[i] = date

sub_df['session'] = session_list

In [84]:
sub_df

,session,semester,subject,depression,anxiety,stress
0,2014-0922,1,1,18,20,38
1,2014-1103,1,1,28,28,30
2,2014-1201,1,1,34,34,32
3,2014-1215,1,1,26,24,22
4,2014-1229,1,1,36,26,30
...,...,...,...,...,...,...
166,2015-0506,2,26,0,0,6
167,2015-0527,2,26,0,0,0
168,2015-0603,2,26,0,0,0
169,2015-0610,2,26,0,0,0


In [4]:
def find_common_channels():
    '''
    Find common channels across all samples
    '''
    channels = sio.loadmat('./data/ch_lib.mat')
    
    channel_all_list = [channels['ch_lib'][0][0][0][i_channel][0] for i_channel in range(30)]

    for i_sub in range(len(channels['ch_lib'])):
    
        if len(channels['ch_lib'][i_sub][0][0])==0:
            continue
        
        channel_list = [channels['ch_lib'][i_sub][0][0][i_channel][0] for i_channel in range(len(channels['ch_lib'][i_sub][0][0]))]
        for channel in channel_all_list:
            if channel not in channel_list:
                channel_all_list.remove(channel)

    return channel_all_list

In [17]:
from os import path
pd.set_option('mode.chained_assignment','warn')

label_format = 3
common_flag = True

channels = sio.loadmat('./data/ch_lib.mat')
channels_common = find_common_channels()
max_num_channel = len(channels_common) if common_flag else 30

# Read dataframes of DASS
df_DASS = pd.read_csv('./data/SFC_DASS21.csv')
df_DASS.loc[:,'session'] = [date.replace("'",'') for date in df_DASS['session']] # Remove ' in session
df_DASS.loc[:,'DASS_record_number'] = list(range(1,len(df_DASS)+1))
group_DASS = df_DASS.groupby(by='subject')
mean_DASS = group_DASS.mean()
std_DASS = group_DASS.std()

# Add corresponding channels in DASS dataframe
channels_list = []
for i_sample in range(len(df_DASS)):
    
    # Select channels
    channels_i = channels['ch_lib'][i_sample][0][0]
    channels_i_name = [channels_i[i][0] for i in range(len(channels_i))]

    channels_list.append(channels_i_name)
df_DASS.loc[:,'channels'] = channels_list

# Read dataframes of DSS
subjects = list(range(1,27))
df_DSS = pd.DataFrame()

for subject in subjects:
    if subject <= 18:
        file_path = './data/DSS/first semester/%s/first_datebystress%d.txt'%(str(subject).zfill(2), subject)
    else:
        file_path = './data/DSS/second semester/%s/second_datebystress%d.txt'%(str(subject-18).zfill(2), subject-18)

    df = pd.read_csv(file_path, sep = '\s+', header=0, names=['session','stress_DSS'], index_col=False)
    df.loc[:,'subject'] = [subject]*len(df)
    df_DSS = df_DSS.append(df)

df_DSS = df_DSS.reset_index(drop=True)
group_DSS = df_DSS.groupby(by='subject')
mean_DSS = group_DSS.mean()
std_DSS = group_DSS.std()
# Take average of stress if some subjects have DSS record on the same date
df_DSS = df_DSS.groupby(by=['subject', 'session']).mean().reset_index()

# Merge dataframes of DASS and recording system
df_summary = pd.read_csv('./data/summary_NCTU_RWN-SFC.csv')
df_summary = df_summary.dropna(how='all')
df_summary = df_summary.rename(columns = {'folder/session':'session', 'labelID':'subject'})
df_summary.loc[:,'subject'] = [int(df_summary['subject'].values[i][1:]) for i in range(len(df_summary['subject']))]
df_all = pd.merge(df_DASS, df_summary, how='inner', sort=False, on=['subject','session'])

# Change session, e.g. 20150506_s26 -> 2015-0506
df_all.loc[:,'session'] = [date.split('_')[0][:4] + '-' + date.split('_')[0][4:] for date in df_all['session']]

# Merge dataframes of DSS and df_all
df_all = pd.merge(df_all, df_DSS, how='inner', sort=False, on=['subject','session'])

labels = []
EEG_list = []
drop_list = []
channels_list = []

for i_sample in range(len(df_all)):
    file_path = './data/rawdata/%d.mat'%(df_all.loc[i_sample,'DASS_record_number'])

    # Continue if file of sample doesn't exist
    if not path.exists(file_path):
        drop_list.append(i_sample)
        continue

    # Get increased stress or normal label for each subject
    sub = df_all.loc[i_sample]['subject']
    stress_DASS = df_all.loc[i_sample]['stress']
    stress_DSS = df_all.loc[i_sample]['stress_DSS']

    if label_format==1:
        threshold = mean_DASS.loc[sub]['stress'] + std_DASS.loc[sub]['stress']
        labels.append(stress_DASS>threshold)
    elif label_format==2:
        labels.append(stress_DASS>15)
    elif label_format==3:
        threshold = mean_DSS.loc[sub]['stress_DSS'] + std_DSS.loc[sub]['stress_DSS']
        labels.append(stress_DSS>threshold)

    # Load EEG
    EEG = sio.loadmat(file_path)
    EEG = EEG['tmp']

    # Select channels
    channels_i = df_all.iloc[i_sample]['channels']
    channels_i_index = [i for i in range(len(channels_i)) if (channels_i[i] in channels_common or not common_flag)]
    channels_i_name = [channels_i[i] for i in range(len(channels_i)) if (channels_i[i] in channels_common or not common_flag)]

    channels_list.append(channels_i_name)
    EEG_list.append(EEG[channels_i_index,:])

df_all = df_all.drop(drop_list)
df_all = df_all.reset_index(drop=True)

if common_flag:
    df_all['channels'] = channels_list

labels = np.asarray(labels, 'int')


/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/pandas/core/dtypes/missing.py:495: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val = np.array(val, copy=False)
/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


ValueError: Must have equal len keys and value when setting with an ndarray

In [18]:
df_all.loc[:,'channels']

0      [C3, C4, CP3, CP4, CPZ, CZ, F3, F4, F7, F8, FC...
1      [C3, C4, CP3, CP4, CPZ, CZ, F3, F4, F7, F8, FC...
2      [C3, C4, CP3, CP4, CPZ, CZ, F3, F4, F7, F8, FC...
3      [C3, C4, CP3, CP4, CPZ, CZ, F3, F4, F7, F8, FC...
4      [C3, C4, CP3, CP4, CPZ, CZ, F3, F4, F7, F8, FC...
                             ...                        
125    [C3, C4, CP3, CP4, CPZ, CZ, F3, F4, F7, F8, FC...
126    [C3, C4, CP3, CP4, CPZ, CZ, F3, F4, F7, F8, FC...
127    [C3, C4, CP3, CP4, CPZ, CZ, F3, F4, F7, F8, FC...
128    [C3, C4, CP3, CP4, CPZ, CZ, F3, F4, F7, F8, FC...
129    [C3, C4, CP3, CP4, CPZ, CZ, F3, F4, F7, F8, FC...
Name: channels, Length: 130, dtype: object

In [19]:
df_all['channels'] = channels_list

In [20]:
df_all['channels']

0      [C4, CP3, CP4, F7, F8, FC4, FZ, P3]
1      [C4, CP3, CP4, F7, F8, FC4, FZ, P3]
2      [C4, CP3, CP4, F7, F8, FC4, FZ, P3]
3      [C4, CP3, CP4, F7, F8, FC4, FZ, P3]
4      [C4, CP3, CP4, F7, F8, FC4, FZ, P3]
                      ...                 
125    [C4, CP3, CP4, F7, F8, FC4, FZ, P3]
126    [C4, CP3, CP4, F7, F8, FC4, FZ, P3]
127    [C4, CP3, CP4, F7, F8, FC4, FZ, P3]
128    [C4, CP3, CP4, F7, F8, FC4, FZ, P3]
129    [C4, CP3, CP4, F7, F8, FC4, FZ, P3]
Name: channels, Length: 130, dtype: object